### Clip Feature Class using as clip features each file of one folder

In [ ]:
in_shp = '/home/jasp/mystuff/dgt/AreasEdificadas2018.shp'

clipshps = '/home/jasp/mystuff/dgt/ref_miss'

outfolder = '/home/jasp/mystuff/dgt/cae_parts'

bname = 'cae'

In [ ]:
import os
import pandas as pd

from glass.g.wenv.grs import run_grass
from glass.pys.oss import lst_ff, fprop
from glass.g.prop.prj import get_epsg

In [ ]:
# List clip shapes and get their id's

# assuming file id is the last part of the filename
# {filename}_{id}.shp
# id must be an integer

cshps = pd.DataFrame([{
    'fid' : int(f.split('.')[0].split('_')[-1]),
    'shp' : f
} for f in lst_ff(
    clipshps, rfilename=True, file_format='.shp'
)])

In [ ]:
# Start GRASS GIS Session

loc = f'loc_{os.path.basename(outfolder).replace("_", "")}'
gb = run_grass(outfolder, location=loc, srs=get_epsg(in_shp))

import grass.script.setup as gs

gs.init(gb, outfolder, loc, 'PERMANENT')

In [ ]:
from glass.g.it.shp import shp_to_grs, grs_to_shp
from glass.g.wenv.grs import shp_to_region
from glass.g.gp.ovl import grsclip

In [ ]:
# Import in_shp
ingrsv = shp_to_grs(in_shp, fprop(in_shp, 'fn'))

# Set region
shp_to_region(ingrsv, 10)

In [ ]:
# Run clip

for i, row in cshps.iterrows():
    # Import clip features
    cfeat = shp_to_grs(
        os.path.join(clipshps, row.shp),
        fprop(row.shp, 'fn'))
    
    # Clip
    clipfeat = grsclip(ingrsv, cfeat, f'{bname}_{str(row.fid)}', cmd=True)
    
    # Export
    grs_to_shp(clipfeat, os.path.join(
        outfolder, f'{clipfeat}.shp'
    ), 'area')